# Homework

1. Complete the Python implementation of the backpropagation exercise in the **Backpropagation** section here above (cell `# try it in Python as homework!`)
    - Create the calculations for obtaining $y$ in PyTorch **using only PyTorch methods and routines**
    - Calculate the gradient
    - Check the values of the gradients and see if it is correct w.r.t. the manual calculations
2. Given the multilayer perceptron defined during the exercises from lab 1:
    - Create 10 random datapoints (with any function you wish, it can be `rand`, `randn`...) and feed them into the network
    - Given the output, calculate the Cross-Entropy loss with respect to the ground truth $[1,2,3,4,1,2,3,4,1,2]$ (classes from 1 to 4). Cross-Entropy loss:
        
        $$ CE(\mathbf{y}, \hat{\mathbf{y}}) = - \sum_{i=1}^{10} \hat{y}_i \log(y_i)$$
        
        where $y_i$ is the one-hot encoding of the $i$-th datapoint. For instance, $y_1 = [1,0,0,0]$.
        **_Note: there is an extremely handy PyTorch function for getting a one-hot encoding out of a vector, so don't try anything fancy._**
    - Backpropagate the error along the network and inspect the gradient of the parameters connecting the input layer and the first hidden layer.
3. Execute the python script `utils/randomized_backpropagation_formula.py`. This creates a formula $f(\mathbf{x})$ with randomized operators and values. Create the computational graph from this formula, do (by hand) the forward pass, then calculate (by hand) $\nabla f(\mathbf{x})$ using the backward gradient computation. Do the same calculation on PyTorch to check the correctness of your calculations. _Note: The formula created by this script is linked to your name and surname, which you have to input before_. The solution to this exercise _should_ be submitted as a scan/good quality picture of a piece of paper (or you can do it on a touch screen and submit the image...), but other formats are acceptable as well.

In [2]:
import torch

## Exercise 1

**Backpropagation**

Let us suppose we have the following calculation

$\mathbf{x} = [1,~2,~-1,~3,~5]$

$ y = f(\mathbf{x}) = \log\{[\exp (x_1 * x_2 )]^2 + \sin (x_3 + x_4 + x_5) \cdot x_5\}$

Find

$\nabla f(\mathbf{x})$

### Part 1

In [3]:
x1 = torch.tensor([1.0], requires_grad=True)
x2 = torch.tensor([2.0], requires_grad=True)
x3 = torch.tensor([-1.0], requires_grad=True)
x4 = torch.tensor([3.0], requires_grad=True)
x5 = torch.tensor([5.0], requires_grad=True)

print(x1)
print(x2)
print(x3)
print(x4)
print(x5)

# x = torch.tensor([1,2,-1,3,5], dtype=torch.float32, requires_grad=True)
# print(x)

tensor([1.], requires_grad=True)
tensor([2.], requires_grad=True)
tensor([-1.], requires_grad=True)
tensor([3.], requires_grad=True)
tensor([5.], requires_grad=True)


In [4]:
a = torch.matmul(x1, x2)
print(a)
b = x3 + x4 + x5
print(b)
c = a.exp()
print(c)
d = torch.pow(c, 2)
print(d)
g = b.sin()
print(g)
h = torch.matmul(g, x5)
print(h)
i = d + h
print(i)
y = torch.log(i)
print("\n")
print("The value of function f(x) evaluated in x is:")
print(y)

tensor(2., grad_fn=<DotBackward0>)
tensor([7.], grad_fn=<AddBackward0>)
tensor(7.3891, grad_fn=<ExpBackward0>)
tensor(54.5982, grad_fn=<PowBackward0>)
tensor([0.6570], grad_fn=<SinBackward0>)
tensor(3.2849, grad_fn=<DotBackward0>)
tensor(57.8831, grad_fn=<AddBackward0>)


The value of function f(x) evaluated in x is:
tensor(4.0584, grad_fn=<LogBackward0>)


### Part 2

In [5]:
y.backward()
print(x1.grad)
print(x2.grad)
print(x3.grad)
print(x4.grad)
print(x5.grad)

tensor([3.7730])
tensor([1.8865])
tensor([0.0651])
tensor([0.0651])
tensor([0.0765])


### Part 3

From manual calculations performed in class, we expected:

- $\partial f/\partial x_1 = 4.14$
- $\partial f/\partial x_2 = 2.07$
- $\partial f/\partial x_3 = 0.08$
- $\partial f/\partial x_4 = 0.08$
- $\partial f/\partial x_4 = 0.093$

As you can see from previous cell in Part 2, the manual calculations are slightly different since we made use of approximations in class.

## Exercise 2

This was the artificial network built in the first assignement:

In [6]:
class MLP(torch.nn.Module):
    def __init__(self, my_bias=False):
        super().__init__()
        self.layers = torch.nn.Sequential(
            torch.nn.Linear(5, 11, bias=my_bias),
            torch.nn.ReLU(),
            torch.nn.Linear(11, 16, bias=my_bias),
            torch.nn.ReLU(),
            torch.nn.Linear(16, 13, bias=my_bias),
            torch.nn.ReLU(),
            torch.nn.Linear(13, 8, bias=my_bias),
            torch.nn.ReLU(),
            torch.nn.Linear(8, 4, bias=my_bias),
            torch.nn.Softmax(dim=1)
        )

    def forward(self, X):
        return self.layers(X)

The following is an instance of it:

In [7]:
model = MLP()
model

MLP(
  (layers): Sequential(
    (0): Linear(in_features=5, out_features=11, bias=False)
    (1): ReLU()
    (2): Linear(in_features=11, out_features=16, bias=False)
    (3): ReLU()
    (4): Linear(in_features=16, out_features=13, bias=False)
    (5): ReLU()
    (6): Linear(in_features=13, out_features=8, bias=False)
    (7): ReLU()
    (8): Linear(in_features=8, out_features=4, bias=False)
    (9): Softmax(dim=1)
  )
)

### Part 1

Feed 10 datapoints to the network:

In [17]:
X = torch.randn(10, 5,requires_grad=True) #Returns a tensor filled with random numbers from a normal distribution with mean 0 and variance 1
print(X)
print("\n")
conf_y_hat = model(X)
print(conf_y_hat)

tensor([[-1.6556e+00,  1.0257e+00,  3.5750e-01, -1.8889e-01, -1.0208e+00],
        [ 5.8702e-02, -7.0612e-01, -1.9208e-01,  3.1658e-02,  1.0980e-01],
        [ 2.1394e-01,  8.8203e-01,  1.6487e+00,  1.2860e+00, -6.9151e-01],
        [-1.3292e-03,  5.7954e-01, -2.7523e-01, -1.0175e+00, -4.9845e-01],
        [ 2.7703e-01,  1.4546e-01,  6.9313e-01,  5.8724e-01,  1.1743e-01],
        [-1.8104e+00, -5.4586e-01, -7.4822e-01, -1.2569e+00, -3.9539e-01],
        [ 1.6971e+00, -5.1476e-01,  1.6629e+00,  3.3935e-01,  1.6295e+00],
        [-3.8369e-01,  1.2811e+00, -1.1413e-01, -7.1252e-01,  6.9003e-01],
        [ 4.0194e-01, -1.9318e+00,  1.2945e+00,  5.1953e-02, -2.2239e+00],
        [-1.7997e+00,  6.1693e-01, -3.5466e-01,  9.1145e-01, -8.8045e-01]],
       requires_grad=True)


tensor([[0.2466, 0.2530, 0.2487, 0.2516],
        [0.2491, 0.2506, 0.2500, 0.2502],
        [0.2452, 0.2540, 0.2485, 0.2522],
        [0.2484, 0.2499, 0.2512, 0.2506],
        [0.2474, 0.2524, 0.2492, 0.2510],
        [0

### Part 2

The predicted classes for the 10 points are then:

In [18]:
y_hat = torch.argmax(conf_y_hat, dim=1) + 1# in order to be confrontable with the ground_truth we add 1 
y_hat

tensor([2, 2, 2, 3, 2, 2, 2, 3, 2, 2])

Define the ground truth:

In [19]:
# We keep it with float point since the torch.nn.functional.one_hot function doesn't allow integers
ground_truth = torch.tensor([1,2,3,4,1,2,3,4,1,2]) # dtype=torch.int) or ground_truth.int() in case
ground_truth

tensor([1, 2, 3, 4, 1, 2, 3, 4, 1, 2])

Define the Cross-Entropy Loss:
$$ CE(\mathbf{y}, \hat{\mathbf{y}}) = - \sum_{i=1}^{10} \hat{y}_i \log(y_i)$$
        
where $y_i$ is the one-hot encoding of the $i$-th datapoint. For instance, $y_1 = [1,0,0,0]$.

**_Note: there is an extremely handy PyTorch function for getting a one-hot encoding out of a vector, so don't try anything fancy._**

In [20]:
def ce_loss(y_hat, y):
    # First we need to transform y in one-hot encoding tensor of shape 4 x 10.
    # To do it we must rescale the ground_truth values in compliance with the function torch.nn.functional.one_hot,
    # since in order to work num_classes must be greater than the biggest values among ground_truth.
    scaled_truth = y-1
    # Then we apply the function one_hot (if num_classes=-1 then num_classes=max_value in ground_truth + 1)
    # and taking the transpose for the matrix multiplication:
    y_hot = torch.nn.functional.one_hot(scaled_truth, num_classes=-1)
    loss = (-1) * ((y_hot.float() @ y_hat.T.log())).sum()
    return loss

Now let's evaluate it:

In [21]:
loss = ce_loss(conf_y_hat, ground_truth)
loss.item()

138.65863037109375

### Part 3

In [22]:
loss.backward()
print(X.grad)

tensor([[-1.2386e-03, -6.5058e-04,  1.6550e-03,  1.0973e-03, -8.4768e-04],
        [-8.5306e-04, -1.8414e-03,  4.0025e-03,  3.2193e-03,  4.5485e-03],
        [-6.3291e-05, -2.3298e-04,  8.9059e-04,  1.8697e-03, -1.7991e-03],
        [-2.4466e-03, -9.8706e-05,  2.3867e-03, -7.8037e-03,  1.0262e-05],
        [-4.3372e-03, -3.3403e-03,  2.3361e-03,  2.7897e-03,  5.9572e-04],
        [-1.1788e-03, -6.7659e-04,  1.0574e-03,  1.8217e-03, -1.3845e-03],
        [ 2.7041e-05, -9.5241e-04,  1.8670e-03,  3.9108e-05,  7.1852e-04],
        [ 1.6376e-03,  1.6944e-03,  2.3387e-03, -6.0555e-03, -1.0176e-03],
        [-2.9756e-03, -9.7615e-04,  2.5698e-04,  1.7374e-03, -1.8373e-03],
        [-1.8865e-03, -1.6928e-03,  1.4776e-03,  1.2648e-03, -9.2947e-04]])


## Exercise 3

Input your name, then press ENTER: Valeria Insogna

f(X) =  exp((sin(x1 + x2) / ReLU(x3 + x4)) - x5)

Your values
{'x1': 3, 'x2': 4, 'x3': -2, 'x4': -1, 'x5': -2}

Calculate ∇f(X) [NB: if division by 0, change the value(s) of X responsible for that]

In [14]:
x_1 = torch.tensor([3.0], requires_grad=True)
x_2 = torch.tensor([4.0], requires_grad=True)
x_3 = torch.tensor([2.0], requires_grad=True) #I have change it from -2 to +2 otherwise there is a  division with denominator 0! 
x_4 = torch.tensor([-1.0], requires_grad=True)
x_5 = torch.tensor([-2.0], requires_grad=True)

print(x_1)
print(x_2)
print(x_3)
print(x_4)
print(x_5)

tensor([3.], requires_grad=True)
tensor([4.], requires_grad=True)
tensor([2.], requires_grad=True)
tensor([-1.], requires_grad=True)
tensor([-2.], requires_grad=True)


Let's first evaluate the forward function step by step:

In [15]:
a = x_1 + x_2
print(a)
b = x_3 + x_4
print(b)
c = a.sin()
print(c)
d = torch.nn.functional.relu(b)
print(d)
e = c/d
print(e)
f = e - x_5
print(f)
g = f.exp()
print(g)


tensor([7.], grad_fn=<AddBackward0>)
tensor([1.], grad_fn=<AddBackward0>)
tensor([0.6570], grad_fn=<SinBackward0>)
tensor([1.], grad_fn=<ReluBackward0>)
tensor([0.6570], grad_fn=<DivBackward0>)
tensor([2.6570], grad_fn=<SubBackward0>)
tensor([14.2533], grad_fn=<ExpBackward0>)


This is the image of the manual calculations for the backward propagation:

![](ex2.png)

Now let's verify the manual counts:

In [16]:
g.backward()
print(x_1.grad)
print(x_2.grad)
print(x_3.grad)
print(x_4.grad)
print(x_5.grad)

tensor([10.7456])
tensor([10.7456])
tensor([-9.3642])
tensor([-9.3642])
tensor([-14.2533])
